#### Implementing early stopping

In [1]:
import os
import re
import numpy as np
import pandas as pd
from keras.models import Sequential
from keras.layers import Dense, Flatten, InputLayer
from sklearn.preprocessing import LabelEncoder
from tensorflow.python.keras import utils
import keras
import imageio 
from PIL import Image 

2025-06-23 13:14:10.048771: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-06-23 13:14:10.081468: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-06-23 13:14:10.167659: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1750664650.230872   22127 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1750664650.266791   22127 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1750664650.326841   22127 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linkin

In [2]:
# Reading the data
train = pd.read_csv('agedetectiontrain/train.csv')

# Image resizing of test data into single numpy array
temp = []
for img_name in train.ID:
    img_path = os.path.join('agedetectiontrain/Train', img_name)
    img = imageio.imread(img_path)
    img = np.array(Image.fromarray(img).resize((32, 32))).astype('float32')    
    temp.append(img)

train_x = np.stack(temp)

# Normalizing the images
train_x = train_x / 255.

# Encoding the categorical variable to numeric
lb = LabelEncoder()
train_y = lb.fit_transform(train.Class)
train_y = keras.utils.to_categorical(train_y)

/tmp/ipykernel_22127/3381015795.py:8: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning disappear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  img = imageio.imread(img_path)


In [3]:
# Specifying all the parameters we will be using in our network
input_num_units = (32, 32, 3)
hidden_num_units = 500
output_num_units = 3

epochs = 100
batch_size = 512

In [4]:
model = Sequential([
          InputLayer(input_shape=input_num_units),
          Flatten(),
          Dense(units=hidden_num_units, activation='relu'),
          Dense(units=output_num_units, activation='softmax'),
        ])

# Defining parameters like optmizer, loss function and evaluating metric
model.compile(loss='categorical_crossentropy', # 
        optimizer=keras.optimizers.Adam(), # Learning rate and momentum can be passed inside optimizer
        metrics=['accuracy'])

# Defining early stopping callback.
# After epoch 50, training can be stopped if not improved in 'val_loss' is seen

cb_early_stop = keras.callbacks.EarlyStopping(monitor='val_loss', patience=8, min_delta=0.02) 
cb = keras.callbacks.TensorBoard(log_dir='early_stop', write_graph=False) 

history = model.fit(train_x, train_y, epochs=epochs, validation_split=0.2, callbacks=[cb, cb_early_stop])

/home/wolf/anaconda3/envs/deepl-env/lib/python3.11/site-packages/keras/src/layers/core/input_layer.py:27: UserWarning: Argument `input_shape` is deprecated. Use `shape` instead.
  warnings.warn(
2025-06-23 13:14:53.254386: E external/local_xla/xla/stream_executor/cuda/cuda_platform.cc:51] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: UNKNOWN ERROR (303)


Epoch 1/100


2025-06-23 13:14:53.470910: W external/local_xla/xla/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 195674112 exceeds 10% of free system memory.


498/498 ━━━━━━━━━━━━━━━━━━━━ 15s 27ms/step - accuracy: 0.5494 - loss: 1.0982 - val_accuracy: 0.6213 - val_loss: 0.8211
Epoch 2/100
498/498 ━━━━━━━━━━━━━━━━━━━━ 12s 24ms/step - accuracy: 0.6065 - loss: 0.8389 - val_accuracy: 0.6313 - val_loss: 0.8028
Epoch 3/100
498/498 ━━━━━━━━━━━━━━━━━━━━ 12s 24ms/step - accuracy: 0.6145 - loss: 0.8294 - val_accuracy: 0.6263 - val_loss: 0.8001
Epoch 4/100
498/498 ━━━━━━━━━━━━━━━━━━━━ 19s 38ms/step - accuracy: 0.6365 - loss: 0.8035 - val_accuracy: 0.6339 - val_loss: 0.7884
Epoch 5/100
498/498 ━━━━━━━━━━━━━━━━━━━━ 13s 27ms/step - accuracy: 0.6462 - loss: 0.7799 - val_accuracy: 0.6464 - val_loss: 0.7870
Epoch 6/100
498/498 ━━━━━━━━━━━━━━━━━━━━ 13s 25ms/step - accuracy: 0.6442 - loss: 0.7816 - val_accuracy: 0.6615 - val_loss: 0.7654
Epoch 7/100
498/498 ━━━━━━━━━━━━━━━━━━━━ 14s 28ms/step - accuracy: 0.6466 - loss: 0.7698 - val_accuracy: 0.6444 - val_loss: 0.7771
Epoch 8/100
498/498 ━━━━━━━━━━━━━━━━━━━━ 16s 32ms/step - accuracy: 0.6605 - loss: 0.7516 - val_

#### Training stopped on epoch 21 due to `val_loss` not improving from min_delta = 0.02